In [ ]:
import requests
import json
import os
import cv2
from PIL import Image


In [ ]:
# get total number of items
# size: square, medium, large
def get_species(species_name, size):
    x = requests.get(f"https://api.inaturalist.org/v1/observations?photos=true&q={species_name}&search_on=names&quality_grade=research&order=desc&order_by=created_at&per_page=100&page=1")
    x = x.json()

    ct = int(x['total_results']) # counter for number of images to collect
    curr_page = 1

    l = 0
    urls = []
    if not os.path.exists(f'images/{species_name}'):
        os.makedirs(f'images/{species_name}')
    while len(x['results']) != 0:
        print(len(x['results'])) # 100 results per page
        for result in x['results']:
            res = result['photos']
            for i in range(len(res)):
                res[i] = res[i]['url']
                # res[i] = res[i].index("square")
                res[i] = res[i].replace("square", size)
                urls.append(res[i])
                file = open(f'images/{species_name}/{res[i].split("/")[-2]}-{res[i].split("/")[-1]}', 'wb')
                file.write(requests.get(res[i]).content)
                file.close()
            ct-=1
        # get the next page
        curr_page+=1
        x = requests.get(f"https://api.inaturalist.org/v1/observations?photos=true&q={species_name}&search_on=names&quality_grade=research&order=desc&order_by=created_at&per_page=100&page={curr_page}")
        x = x.json()
    return urls

    

In [ ]:
# Microcystis%20aeruginosa
urls = get_species("Microcystis aeruginosa", "original")
print(len(urls))
print(urls[:5])

In [ ]:
di = {}
for file in os.listdir(f'images/Microcystis aeruginosa'):
    print(file)
    img = cv2.imread(f'images/Microcystis aeruginosa/{file}')
    if img is None:
        di['None'] = di.get('None', 0) + 1
    else:
        di[img.shape] = di.get(img.shape, 0) + 1

In [ ]:
print(di)

In [ ]:
# CROPPING FOR NOW BUT IN THE FUTURE MAYBE USE CROPPED SEGMENTS?
# print(cv2.imread('images/Microcystis aeruginosa/2218866-medium.jpg').shape)
def crop_center(pil_img, crop_width, crop_height):
    img_width, img_height = pil_img.size
    return pil_img.crop(((img_width - crop_width) // 2,
                         (img_height - crop_height) // 2,
                         (img_width + crop_width) // 2,
                         (img_height + crop_height) // 2))

def resize_images(name):
    di = {}
    for file in os.listdir(f'images/{name}'):
        print(file)
        img = cv2.imread(f'images/{name}/{file}')
        if img is None:
            di['None'] = di.get('None', 0) + 1
        else:
            image = Image.open(f'images/{name}/{file}')
            image.save('original.jpg')
            # crop image to a square
            width, height = image.size
            # resize to the smallest dimension out of width & height to maintain square
            image = crop_center(image, min(width, height), min(width, height))
            
            image = image.resize((416, 416))

            # save the resized image
            image.save('resized.jpg')
            break


In [ ]:
resize_images("Microcystis aeruginosa")